# Seattle AirBnB data

## Using data to understand the homeowner's market in Seattle

I approached the data as if I were a homeowner Seattle. If I were a homeowner in Seattle, my main objective would be to offer a great experience for my guests while maximizing revenues. Hence, I structured my business understanding questions around these objectives. My questions for my analysis are thus as follows: 

### 1. Business Understanding:

##### 1. Can we predict what drives higher ratings?

If I were a Seattle home owner, I would find out if data could inform what would lead to higher ratings. In using data to answer this question, I would be able to determine if I could craft an experience that will ensure I get high ratings.

##### 2. When and where are the most popular times of the year for rentals of Seattle homes?

As an aspiring home owner, I would like to know if there were particular areas that are popular among guests, as well as the most popular times of the year so that I could consider renting out homes in popular areas while being prepared for those times. 

##### 3. When and where are the highest revenue-generating times of the year for Seattle home-owners?

In order to set prices competitively, it would be useful to know when are the highest prices for rentals, as well as where are the highest priced areas.

### 2. Data Understanding

### 2.1 Data Exploration

All data was obtained from Kaggle: https://www.kaggle.com/airbnb/seattle/home

In [463]:
#import libraries and load data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython import display
import collections
from itertools import chain
import sklearn
from time import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
%matplotlib inline

listings = pd.read_csv('./seattle/listings.csv')
calendar = pd.read_csv('./seattle/calendar.csv')
reviews = pd.read_csv('./seattle/reviews.csv')

In [464]:
#explore columns in datasets
print(listings.columns.values)
print(calendar.columns.values)
print(reviews.columns.values)

['id' 'listing_url' 'scrape_id' 'last_scraped' 'name' 'summary' 'space'
 'description' 'experiences_offered' 'neighborhood_overview' 'notes'
 'transit' 'thumbnail_url' 'medium_url' 'picture_url' 'xl_picture_url'
 'host_id' 'host_url' 'host_name' 'host_since' 'host_location'
 'host_about' 'host_response_time' 'host_response_rate'
 'host_acceptance_rate' 'host_is_superhost' 'host_thumbnail_url'
 'host_picture_url' 'host_neighbourhood' 'host_listings_count'
 'host_total_listings_count' 'host_verifications' 'host_has_profile_pic'
 'host_identity_verified' 'street' 'neighbourhood'
 'neighbourhood_cleansed' 'neighbourhood_group_cleansed' 'city' 'state'
 'zipcode' 'market' 'smart_location' 'country_code' 'country' 'latitude'
 'longitude' 'is_location_exact' 'property_type' 'room_type'
 'accommodates' 'bathrooms' 'bedrooms' 'beds' 'bed_type' 'amenities'
 'square_feet' 'price' 'weekly_price' 'monthly_price' 'security_deposit'
 'cleaning_fee' 'guests_included' 'extra_people' 'minimum_nights'
 'm

It appears that all the datasets can potentially be merged by their listing ID, if needed during analysis. First, check that all columns are variables and rows are individuals.

In [465]:
#check no. of rows and columns
print(listings.shape)
print(calendar.shape)
print(reviews.shape)

(3818, 92)
(1393570, 4)
(84849, 6)


In [466]:
#check for missing values in the columns for each dataset, get percentages
(listings.isnull().sum()/len(listings)).sort_values(ascending=False)

license                             1.000000
square_feet                         0.974594
monthly_price                       0.602672
security_deposit                    0.511262
weekly_price                        0.473808
notes                               0.420639
neighborhood_overview               0.270299
cleaning_fee                        0.269775
transit                             0.244631
host_about                          0.224987
host_acceptance_rate                0.202462
review_scores_accuracy              0.172342
review_scores_checkin               0.172342
review_scores_value                 0.171818
review_scores_location              0.171556
review_scores_cleanliness           0.171032
review_scores_communication         0.170508
review_scores_rating                0.169460
reviews_per_month                   0.164222
first_review                        0.164222
last_review                         0.164222
space                               0.149031
host_respo

In [467]:
#check distribution of null values
(listings.isnull().sum()/len(listings)).describe()

count    92.000000
mean      0.084893
std       0.181492
min       0.000000
25%       0.000000
50%       0.000000
75%       0.136983
max       1.000000
dtype: float64

For the listing dataset, it looks as though there are a number of columns containing missing values. The license column is completely null.

Looking at the spread of null values, we can see that the average % of null rows in each column is around 10-11%, with 75% of columns having 17% of rows being missing. As I would prefer not to drop columns unnecessarily, I would consider drop columns with more than 30% rows that are null.

In [468]:
(calendar.isnull().sum()/len(calendar)).sort_values(ascending=False)

price         0.32939
available     0.00000
date          0.00000
listing_id    0.00000
dtype: float64

For the calendar dataset, the price column has 32% of rows containing null values.

In [469]:
(reviews.isnull().sum()/len(reviews)).sort_values(ascending=False)

comments         0.000212
reviewer_name    0.000000
reviewer_id      0.000000
date             0.000000
id               0.000000
listing_id       0.000000
dtype: float64

The reviews dataset has almost no missing values.

In order to prepare the data for the 3 business questions, we need to look at the 3 datasets and determine which datasets and columns contained within them that are relevant to the question above.

We have 3 datasets: listings, calendar and reviews. Based on our brief exploration above, we can see that the dataset most relevant to our analysis for this question is the listings dataset. The calendar dataset looks to be more relevant to supplement the listings dataset for our 2nd question on popular times and availability. 

Meanwhile, the reviews dataset is more relevant for qualitative predictors and is mainly unstructured data, hence we will only analyse it if we lack sufficient information to answer our questions.

After determining the datasets that are relevant for answering our questions, we move to preparing the data for our analysis.

### 2.2: Business Question 1: Can we predict what drives higher ratings?
#### 2.2.1 Data Preparation

#### 2.2.1.a Treatment of missing values

Seeing as there are many missing values in the license column, and it is not relevant to the questions above, we can drop it from our analysis dataset. As mentioned above, I would also drop columns that are more than 30% of missing rows, if there are not too many and we don't foresee losing too much information.

In [470]:
#drop license column
listings.drop(columns=['license'],inplace=True)

In [471]:
#check columns >=30% missing values
[cols for cols in listings.columns.values if (listings[cols].isnull().sum()/len(listings))>=0.30]

['notes', 'square_feet', 'weekly_price', 'monthly_price', 'security_deposit']

Considering the columns above, we know that there is the price column that would contain similar information, so we can drop the weekly_price and monthly_price columns without worrying about losing information. 

For the security_deposit column, the missing values most likely means there is no charge, so we can replace those values with 0.

In [472]:
#fill missing values with 0
listings['security_deposit']=listings['security_deposit'].fillna(0)

In [473]:
#drop weekly_price, monthly_price columns
listings.drop(columns=['weekly_price','monthly_price'],inplace=True)

As for notes, it appears to be unstructured data. Meanwhile, for square_feet, the number of missing values are so high that I will just drop it.

In [474]:
#check values in notes columns
print(listings['notes'].head(10))
#drop square_feet variables
listings.drop(columns=['notes','square_feet'],inplace=True)

0                                                  NaN
1    What's up with the free pillows?  Our home was...
2    Our house is located just 5 short blocks to To...
3                                                  NaN
4                                            Belltown 
5    Let me know if you need anything or have sugge...
6    The room now has a mini frig to keep your favo...
7    There are three rentals in our back yard . If ...
8                                                  NaN
9    What's up with the free pillows?  Our home was...
Name: notes, dtype: object


Next we assess to see how many rows containing missing values.

In [475]:
# How much data is missing in each row of the dataset?

missing=pd.DataFrame({'null':listings.isnull().sum(axis=1)})
missing.reset_index(inplace=True)
missing.rename(columns={'index': 'rows'}, inplace=True)
missing.sort_values('null',ascending=False).head(10)

,rows,null
1419,1419,29
1297,1297,25
2943,2943,23
2671,2671,22
98,98,22
2585,2585,21
3,3,21
3677,3677,21
1014,1014,20
3662,3662,20


In [476]:
missing.describe()

,rows,null
count,3818.000000,3818.000000
mean,1908.500000,3.827135
std,1102.305992,4.994624
min,0.000000,0.000000
25%,954.250000,0.000000
50%,1908.500000,2.000000
75%,2862.750000,4.000000
max,3817.000000,29.000000


Based on the spread of missing values across rows, we can see that the missing values in majority of the rows are quite minimal, with mean of null values around 4-5. However, some rows have up to 30 missing values. This is up to a third of values that are missing in the columns. However, since the number of rows in the dataset are quite small, only nearly 4000, I would rather not drop those rows, and impute them later.

#### 2.2.1.b Choosing the target (y) variable
Next, we revisit the question, which is on driving higher ratings in homes. The relevant column that can be seen as the target variable (y column) would be in the set of review_scores columns. However, we can see that there are several columns in the review_scores.

In [477]:
#check column names that begin with 'review_scores' 
[col for col in listings if col.startswith('review_scores_')]

['review_scores_rating',
 'review_scores_accuracy',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value']

In [478]:
#check type of values in column names that begin with 'review_scores' 
[type(listings[col][0]) for col in listings if col.startswith('review_scores_')]

[numpy.float64,
 numpy.float64,
 numpy.float64,
 numpy.float64,
 numpy.float64,
 numpy.float64,
 numpy.float64]

Based on [AirBnB's ratings methodology](https://www.airbnb.com/help/article/1257/how-do-star-ratings-work), the overall experience is the one that determines the overall experience for guests, and so the review_scores_rating column is the one I would set as my target variable. And the value type is a float, so we can attempt to predict the numeric value.

Since this is the target variable, we need to ensure that the target column contains values in both the training and test datasets. 

In [479]:
#check missing values in review_scores_rating column
listings['review_scores_rating'].isnull().sum()

647

As there are 647 rows of missing values in the column, we should drop these rows from our dataset.

In [480]:
#drop rows that are missing review_scores_rating
listings = listings.dropna(axis=0, subset=['review_scores_rating'])

In [481]:
# Split the data into features and target label
target = listings['review_scores_rating']
features = listings.drop('review_scores_rating', axis = 1)

As the id column doesn't help with prediction, we can also drop it from the dataset.

In [482]:
features.drop(columns='id',inplace=True)

#### 2.2.1.c Feature engineering

However, we need to revisit the other columns in the listings dataset. There are quite a few redundant columns that are unnecessary.

For example, it is unnecessary to have columns that only contain one unique value as they don't provide any predictive power.

In [483]:
#find columns in dataset that only contain one unique value
one_unique=[col for col in features.columns.values if features[col].nunique()==1]
one_unique

['scrape_id',
 'last_scraped',
 'experiences_offered',
 'state',
 'market',
 'country_code',
 'country',
 'has_availability',
 'calendar_last_scraped',
 'requires_license',
 'jurisdiction_names']

Any columns that contain 'url' in the name are also irrelevant as they contain no predictive power or characteristics that lead to higher ratings for homes.

In [484]:
#find columns containing 'url' in the name
url_col=[col for col in features.columns.values if 'url' in col]
url_col

['listing_url',
 'thumbnail_url',
 'medium_url',
 'picture_url',
 'xl_picture_url',
 'host_url',
 'host_thumbnail_url',
 'host_picture_url']

In [485]:
#add url and single unique value to drop columns list
drop_cols=[]
drop_cols.extend(one_unique)
drop_cols.extend(url_col)
#drop columns from dataset
features.drop(columns=drop_cols,inplace=True)

In [486]:
#get total no. of columns after dropping columns
len(features.columns.values)

66

For the remaining columns (ignoring the review_scores/target variable columns), we will need to look into a sample of one row from each column to determine if it would be valuable for our predictions or not, and to determine if further feature engineeringis required for some columns. 

In a way, we need to build our own simple data dictionary for our use, to more easily identify what kind of feature engineering is required.

In [487]:
features_sample=[(x,features[x][1]) for x in features.columns.values]
features_sample=pd.DataFrame(features_sample, columns=['column_name','sample_value'])
features_sample.set_index('column_name',inplace=True)
#check types of values in each column - add new column of value_type to our data dictionary
features_sample['value_type']=[type(x) for x in features_sample['sample_value']]
#check how many unique values are in each column for categorical variables encoding
features_sample['nunique']=[features[col].nunique() for col in features_sample.index.values]
features_sample

,sample_value,value_type,nunique
column_name,,,
name,Bright & Airy Queen Anne Apartment,<class 'str'>,3153
summary,Chemically sensitive? We've removed the irrita...,<class 'str'>,2868
space,"Beautiful, hypoallergenic apartment in an extr...",<class 'str'>,2740
description,Chemically sensitive? We've removed the irrita...,<class 'str'>,3108
neighborhood_overview,"Queen Anne is a wonderful, truly functional vi...",<class 'str'>,2196
transit,"Convenient bus stops are just down the block, ...",<class 'str'>,2271
host_id,5177328,<class 'numpy.int64'>,2288
host_name,Andrea,<class 'str'>,1283
host_since,2013-02-21,<class 'str'>,1264


From our data dictionary above, we can see that there are quite a number of columns with unstructured data in the form of text and sentences, e.g. name, summary, description, neighbourhood_overview, etc. We will decide what to do with them later as there will be quite a lot of engineering to be done on those features.

For the other columns, we can see some that may not be relevant in the predicitive model. For example, the host particulars like host_id, host_name, host_since, host_location, host_about, host_verifications don't look like they will be useful, as they contain mainly unstructured and irrelevant information to our question of interest.

In [488]:
#remove relevant columns, update data dictionary at the same time
drop_cols=['host_id', 'host_name', 'host_since', 'host_location', 'host_about', 'host_verifications']
features.drop(columns=drop_cols,inplace=True)
features_sample.drop(drop_cols,inplace=True)

Furthermore, there are some redundant columns that can be removed as well, for example the neighbourhood data (it looks like they have been aggregated into neighbourhood_group_cleansed which may be more useful).

In [489]:
#check neighbourhood columns to see which one would provide more information
print(features['neighbourhood_cleansed'].unique())
print(features['neighbourhood_group_cleansed'].unique())
print(features['neighbourhood'].unique())

['West Queen Anne' 'Adams' 'West Woodland' 'East Queen Anne' 'Wallingford'
 'North Queen Anne' 'Green Lake' 'Westlake' 'Mann' 'Madrona'
 'University District' 'Harrison/Denny-Blaine' 'Minor' 'Leschi' 'Atlantic'
 'Pike-Market' 'Eastlake' 'South Lake Union' 'Lawton Park' 'Briarcliff'
 'Belltown' 'International District' 'Central Business District'
 'First Hill' 'Yesler Terrace' 'Pioneer Square' 'Gatewood' 'Arbor Heights'
 'Alki' 'North Admiral' 'Crown Hill' 'Fairmount Park' 'Genesee' 'Interbay'
 'Industrial District' 'Mid-Beacon Hill' 'South Beacon Hill' 'Greenwood'
 'Holly Park' 'Fauntleroy' 'North Beacon Hill' 'Brighton' 'South Delridge'
 'View Ridge' 'Dunlap' 'Rainier Beach' 'Columbia City' 'Mount Baker'
 'Seward Park' 'North Delridge' 'Maple Leaf' 'Ravenna' 'Riverview'
 'Portage Bay' 'Bryant' 'Montlake' 'Broadway' 'Loyal Heights'
 'Victory Heights' 'Matthews Beach' 'Whittier Heights' 'Meadowbrook'
 'Olympic Hills' 'Roosevelt' 'Lower Queen Anne' 'Wedgwood'
 'North Beach/Blue Ridge' 'C

After checking the neighbourhood columns above, I decided to use neighbourhood_group_cleansed instead of the other two variables as it provides less number of levels as a categorical variable, while also offering a good amount of information.

In [490]:
#remove redundant columns, update data dictionary at the same time
drop_cols=['neighbourhood','host_neighbourhood','neighbourhood_cleansed']
features.drop(columns=drop_cols,inplace=True)
features_sample.drop(drop_cols,inplace=True)

City, State, Street, smart_location, latitude and longitude are all redundant data as all listings are in Seattle, WA. We have information on location through other variables like neighbourhood and zip code. Thus they can be dropped. The variables first_review and last_review are dates that I am not going to focus on in my analysis for question 1. This also applied to the variable calendar_updated.

In [491]:
#remove redundant data, update data dictionary at the same time
drop_cols=['city','street','smart_location',
            'latitude','longitude','first_review','last_review','calendar_updated']
features.drop(columns=drop_cols,inplace=True)
features_sample.drop(drop_cols,inplace=True)

After removing the 'low-hanging fruit' of irrelevant columns, we need to revisit our data dictionary and check if the values in each column can be processed by the model. 
number of levels they have that are required to be encoded.

In [492]:
print('No. of columns= {} columns.'.format(len(features_sample)))
features_sample

No. of columns= 49 columns.


,sample_value,value_type,nunique
column_name,,,
name,Bright & Airy Queen Anne Apartment,<class 'str'>,3153
summary,Chemically sensitive? We've removed the irrita...,<class 'str'>,2868
space,"Beautiful, hypoallergenic apartment in an extr...",<class 'str'>,2740
description,Chemically sensitive? We've removed the irrita...,<class 'str'>,3108
neighborhood_overview,"Queen Anne is a wonderful, truly functional vi...",<class 'str'>,2196
transit,"Convenient bus stops are just down the block, ...",<class 'str'>,2271
host_response_time,within an hour,<class 'str'>,4
host_response_rate,98%,<class 'str'>,43
host_acceptance_rate,100%,<class 'str'>,2


From our simple data dictionary, we can see that a lot of feature engineering is required for many of the columns. For example, the host_response_rate and host_acceptance_rate are recorded as type strings eventhough they are percentages. The pricing data (e.g. price, security_deposit, etc.) are also recorded as strings, eventhough they would be more useful to the predictive model as numerical values.

Additionally, the columns 'amenities' are also recorded as lists of strings, so we will need to process those as well.

#### 2.2.1.d Encoding for categorical variables 

Leaving all columns containing unstructured data for later, we first encode all categorical variables. Referring to our data dictionary, the categorical variables are as follows:

    host_response_time
    host_is_superhost
    host_has_profile_pic
    host_identity_verified
    neighbourhood_group_cleansed
    zipcode
    is_location_exact
    property_type
    room_type
    bed_type
    amenities (which needs further processing)
    instant_bookable
    cancellation_policy
    require_guest_profile_picture
    require_guest_phone_verification
    security_deposit_flag

I created the security_deposit_flag variable so we can leave that aside. But for the others, we need to encode them according to their levels. With that, I used pandas_getdummies function.

In [493]:
#check zipcode unique values - appears to have a weird value there
print(features['zipcode'].unique())
#convert weird value into 98122
features.loc[features['zipcode'] == '99\n98122', 'zipcode'] = '98122'
print(features['zipcode'].unique())

['98119' '98109' '98107' '98117' nan '98103' '98105' '98115' '98122'
 '98112' '98144' '99\n98122' '98101' '98121' '98102' '98199' '98104'
 '98134' '98136' '98126' '98146' '98116' '98177' '98118' '98108' '98133'
 '98106' '98178' '98125']
['98119' '98109' '98107' '98117' nan '98103' '98105' '98115' '98122'
 '98112' '98144' '98101' '98121' '98102' '98199' '98104' '98134' '98136'
 '98126' '98146' '98116' '98177' '98118' '98108' '98133' '98106' '98178'
 '98125']


In [494]:
#encode categorical variables
binary_cols=['host_is_superhost','host_has_profile_pic','host_identity_verified','is_location_exact'
             ,'instant_bookable','require_guest_profile_picture','require_guest_phone_verification']
features[binary_cols]=np.where(features[binary_cols]=='t', 1, 0)
encode_cols=['host_response_time','neighbourhood_group_cleansed','zipcode'
             ,'property_type','room_type','bed_type','cancellation_policy']
features=pd.get_dummies(data=features, columns=encode_cols,drop_first=True)

#### Feature engineering for 'amenities' column

Next we need to look at the 'amenities' column and engineer it to extract categorical variables. Since the amenities column is in the form of list of amenities, we should extract each amenity and it would be its own categorical feature for each listing.

In [495]:
#get dataframe of features with amenities split up
amenities_df=features['amenities'].str.split(',',expand=True)
amenities_df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,{TV,"""Cable TV""",Internet,"""Wireless Internet""","""Air Conditioning""",Kitchen,Heating,"""Family/Kid Friendly""",Washer,Dryer},...,None,None,None,None,None,None,None,None,None,None
1,{TV,Internet,"""Wireless Internet""",Kitchen,"""Free Parking on Premises""","""Buzzer/Wireless Intercom""",Heating,"""Family/Kid Friendly""",Washer,Dryer,...,None,None,None,None,None,None,None,None,None,None
2,{TV,"""Cable TV""",Internet,"""Wireless Internet""","""Air Conditioning""",Kitchen,"""Free Parking on Premises""","""Pets Allowed""","""Pets live on this property""",Dog(s),...,Shampoo},None,None,None,None,None,None,None,None,None
4,{TV,"""Cable TV""",Internet,"""Wireless Internet""",Kitchen,Heating,"""Family/Kid Friendly""","""Smoke Detector""","""Carbon Monoxide Detector""","""First Aid Kit""",...,None,None,None,None,None,None,None,None,None,None
5,"{""Wireless Internet""","""Free Parking on Premises""",Heating,"""Smoke Detector""",Essentials,Shampoo},None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


We need to clean the amenities dataframe by remove redundant punctuation and getting only unique values to create the categorical features.

In [496]:
#remove punctuation from amenities dataframe
amenities_df = amenities_df.replace('[^\w\s]','', regex=True)

#add all amenities in dataframe to list
amenities_unique=[amenities_df[x].unique().tolist() for x in amenities_df.columns.values]

#unnest list and get unique values of amenities
amenities_unique=set(list(chain.from_iterable(amenities_unique)))
print(amenities_unique)

#remove '' and None values
amenities_unique.remove('')
amenities_unique.remove(None)
print('Removed null values:')
print(amenities_unique)

{'', 'Laptop Friendly Workspace', 'Indoor Fireplace', 'Smoke Detector', 'Washer  Dryer', 'Other pets', 'Internet', 'Air Conditioning', 'Pets Allowed', 'BuzzerWireless Intercom', 'Carbon Monoxide Detector', 'Heating', 'Elevator in Building', 'Dryer', 'Fire Extinguisher', 'Kitchen', 'TV', 'Pets live on this property', 'Cable TV', 'Doorman', 'Hair Dryer', 'Iron', 'Wireless Internet', 'Pool', 'Cats', 'Safety Card', 'Smoking Allowed', 'Hangers', 'Suitable for Events', 'Lock on Bedroom Door', 'FamilyKid Friendly', 'Gym', 'First Aid Kit', 'Essentials', 'Wheelchair Accessible', 'Hot Tub', 'Washer', 'Breakfast', '24Hour Checkin', None, 'Shampoo', 'Free Parking on Premises', 'Dogs'}
Removed null values:
{'Laptop Friendly Workspace', 'Indoor Fireplace', 'Smoke Detector', 'Washer  Dryer', 'Other pets', 'Internet', 'Air Conditioning', 'Pets Allowed', 'BuzzerWireless Intercom', 'Carbon Monoxide Detector', 'Heating', 'Elevator in Building', 'Dryer', 'Fire Extinguisher', 'Kitchen', 'TV', 'Pets live on

In [497]:
#flag listings according to available amenities - note: is there a more efficient way to code this?
for x in amenities_df.columns.values:
    for y in amenities_unique:
        amenities_df[y]= np.where(amenities_df[x]==y, 1, 0)

#drop unnecessary columns - redundant due to new flag columns
amenities_df.drop(columns=[x for x in range(30)],inplace=True)
amenities_df.head(5)

,Laptop Friendly Workspace,Indoor Fireplace,Smoke Detector,Washer Dryer,Other pets,Internet,Air Conditioning,Pets Allowed,BuzzerWireless Intercom,Carbon Monoxide Detector,...,First Aid Kit,Essentials,Wheelchair Accessible,Hot Tub,Washer,Breakfast,24Hour Checkin,Shampoo,Free Parking on Premises,Dogs
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [498]:
#merge amenities columns with original dataset
features=features.join(amenities_df,how='left')

#drop amenities column
features.drop(columns='amenities',inplace=True)
features.head(5)

,name,summary,space,description,neighborhood_overview,transit,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,...,First Aid Kit,Essentials,Wheelchair Accessible,Hot Tub,Washer,Breakfast,24Hour Checkin,Shampoo,Free Parking on Premises,Dogs
0,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,NaN,NaN,96%,100%,0,3.0,...,0,0,0,0,0,0,0,0,0,0
1,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,"Queen Anne is a wonderful, truly functional vi...","Convenient bus stops are just down the block, ...",98%,100%,1,6.0,...,0,0,0,0,0,0,0,0,0,0
2,New Modern House-Amazing water view,New modern house built in 2013. Spectacular s...,"Our house is modern, light and fresh with a wa...",New modern house built in 2013. Spectacular s...,Upper Queen Anne is a charming neighborhood fu...,A bus stop is just 2 blocks away. Easy bus a...,67%,100%,0,2.0,...,0,0,0,0,0,0,0,0,0,0
4,Charming craftsman 3 bdm house,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,We are in the beautiful neighborhood of Queen ...,The nearest public transit bus (D Line) is 2 b...,100%,NaN,0,2.0,...,0,0,0,0,0,0,0,0,0,0
5,Private unit in a 1920s mansion,We're renting out a small private unit of one ...,If you include a bit of your background in you...,We're renting out a small private unit of one ...,This part of Queen Anne has wonderful views an...,The #1 and #2 bus picks up 2-3 blocks away (di...,NaN,NaN,0,1.0,...,0,0,0,0,0,0,0,0,0,0


#### 2.2.1.e Transforming strings into numerical values

Now we look at the numerical features that are currently being encoded as strings. As mentioned above, these include:

    host_response_rate
    host_acceptance_rate
    price
    security_deposit
    cleaning_fee
    extra_people
    
All these variables need to be transformed into numerical values. 

Specifically, since the host_response_rate and host_acceptance rate are percentage values, they can be transformed into integers. Meanwhile, since the pricing data all involve dollar amounts, they should be converted into floats.

In [499]:
#strip columns of percentages
cols=['host_response_rate','host_acceptance_rate']
features[cols] = features[cols].replace('%', '', regex=True).astype(float)

#convert host_response_rate and host_acceptance_rate into integer value percentages
features[cols]=features[cols]/100
features[cols].head(5)

,host_response_rate,host_acceptance_rate
0,0.96,1.0
1,0.98,1.0
2,0.67,1.0
4,1.00,NaN
5,NaN,NaN


In [500]:
#strip columns of dollar signs
cols=['price', 'security_deposit','cleaning_fee','extra_people']
features[cols]=features[cols].replace('[\$,]', '', regex=True).astype(float)
features[cols].head(5)

,price,security_deposit,cleaning_fee,extra_people
0,85.0,0.0,NaN,5.0
1,150.0,100.0,40.0,0.0
2,975.0,1000.0,300.0,25.0
4,450.0,700.0,125.0,15.0
5,120.0,0.0,40.0,30.0


#### 2.2.1 f Final Preprocessing

Now we need to revisit what to do with the unstructured data. After reading around and experimenting with some simple text processing, I decided that the columns do not contain enough useful information to warrant further processing. Moreover, we already have 148 columns which are a large amount of variables. Hence, I will drop these columns from my analysis.

In [501]:
drop_cols=['name','summary', 'space', 'description', 'neighborhood_overview','transit']
features.drop(columns=drop_cols,inplace=True)

Now we check our final dataset for any other missing values that require pre-processing.

In [502]:
features_null=pd.DataFrame({'null':features.isnull().sum(axis=0),
                            'null_percentage (%)':(features.isnull().sum(axis=0)/len(features))*100})
features_null.reset_index(inplace=True)
features_null.rename(columns={'index': 'cols'}, inplace=True)
features_null.sort_values('null_percentage (%)',ascending=False).head(10)

,cols,null,null_percentage (%)
14,cleaning_fee,759,23.935667
1,host_acceptance_rate,477,15.042573
0,host_response_rate,312,9.839167
9,bathrooms,12,0.378430
26,review_scores_checkin,11,0.346894
24,review_scores_accuracy,11,0.346894
29,review_scores_value,9,0.283822
28,review_scores_location,8,0.252286
10,bedrooms,6,0.189215
25,review_scores_cleanliness,6,0.189215


There are still quite a number of columns that contain misisng values - in particular, the review_scores_rating column which is our target column contains 647 missing rows, or 17% missing values. 

We will have need to drop those rows from our training and test set as they will not be able to help with the training the model.

Meanwhile, cleaning_fee requires the same treatment as security_deposit - if it is missing, the value should be 0 as there is no charge.

In [503]:
#fill missing values with 0
features['cleaning_fee']=features['cleaning_fee'].fillna(0)

In [504]:
#check remaining missing values
features_null=pd.DataFrame({'null':features.isnull().sum(axis=0),
                            'null_percentage (%)':(features.isnull().sum(axis=0)/len(features))*100})
features_null.reset_index(inplace=True)
features_null.rename(columns={'index': 'cols'}, inplace=True)
features_null.sort_values('null_percentage (%)',ascending=False).head(10)

,cols,null,null_percentage (%)
1,host_acceptance_rate,477,15.042573
0,host_response_rate,312,9.839167
9,bathrooms,12,0.378430
26,review_scores_checkin,11,0.346894
24,review_scores_accuracy,11,0.346894
29,review_scores_value,9,0.283822
28,review_scores_location,8,0.252286
25,review_scores_cleanliness,6,0.189215
10,bedrooms,6,0.189215
27,review_scores_communication,4,0.126143


In [505]:
#check spread of values for host_acceptance_rate column
features['host_acceptance_rate'].describe()

count    2694.000000
mean        0.999629
std         0.019266
min         0.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: host_acceptance_rate, dtype: float64

For the host_acceptance_rate column, we can see above that the mean is actually around 99.99%, and most of the values are nearly 100%. Hence we can drop the column as it does not offer that much information. In fact, if we refer back to our data dictionary above, there are only 2 unique values.

In [506]:
#drop host_acceptance_rate column
features.drop(columns='host_acceptance_rate',inplace=True)

#### Train-test split and imputation

Before imputing missing values, I split my pre-processed dataset into a training set and test set.

In [551]:
#split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features, 
                                                    target, 
                                                    test_size=0.20, 
                                                    random_state=42)

Now we can do the imputation of the median on all misssing training data.

In [552]:
#impute all missing values with median
imp=Imputer(strategy='median')
X_train_imp=imp.fit_transform(X_train)

In [553]:
#impute all missing test data with median
X_test_imp=imp.transform(X_test)

#### 2.2.2 Modeling

Now we are finally ready to model. I decided to use a simple linear regresison model. As such, I need to scale/normalize the features. 

In [554]:
#fit and transform training data
scaler=StandardScaler()
X_train_imp=scaler.fit_transform(X_train_imp)

#transform test data
X_test_imp=scaler.transform(X_test_imp)

#### 2.2.2 b Implementation of the model

In choosing which model to predict the ratings, I chose a simple linear regression model as a baseline comparison, and a random forest model.

To implement the model, I first create a training and test pipeline. I referred extensively to my [Term 1 project](https://github.com/syuenloh/UdacityDataScienceNanodegreeTerm1/blob/master/projects/p1_charityml/finding_donors.ipynb) on Finding Donors for CharityML, which was done with extensive guidance from Udacity, for the functions and workflow for this section.

In [555]:
def train_predict(learner, sample_size, X_train, y_train, X_test, y_test): 
    '''
    inputs:
       - learner: the learning algorithm to be trained and predicted on
       - sample_size: the size of samples (number) to be drawn from training set
       - X_train: features training set
       - y_train: income training set
       - X_test: features testing set
       - y_test: income testing set
    '''
    results = {}
    
    #Fit the learner to the training data and get training time
    start = time() 
    learner = learner.fit(X_train[:sample_size], y_train[:sample_size])
    end = time() 
    results['train_time'] = end-start
    
    # Get predictions on the test set(X_test), then get predictions on first 300 training samples
    start = time() 
    predictions_test = learner.predict(X_test)
    predictions_train = learner.predict(X_train[:300])
    end = time() 
    
    # Calculate the total prediction time
    results['pred_time'] = end-start
    
    #Compute accuracy on the first 300 training samples
    results['mse_train'] = mean_squared_error(y_train[:300],predictions_train)
    
    #Compute accuracy on test set
    results['mse_test'] = mean_squared_error(y_test,predictions_test)
       
    print("{} trained on {} samples.".format(learner.__class__.__name__, sample_size))
    print("MSE_train: %.4f" % results['mse_train'])
    print("MSE_test: %.4f" % results['mse_test'])
    print("Training score:%.4f" % learner.score(X_train,y_train))
    print("Test score:%.4f" % learner.score(X_test,y_test))
    return results

In [556]:
#Initialize the model
clf = LinearRegression()

# Calculate the number of samples for 1%, 10%, and 100% of the training data
samples_100 = len(y_train)
samples_10 = int(0.10*len(y_train))
samples_1 = int(0.01*len(y_train))

# Collect results on the learners
results = {}
for model in [clf]:
    clf_name = clf.__class__.__name__
    results[clf_name] = {}
    for i, samples in enumerate([samples_100]):
        results[clf_name][i] = \
        train_predict(clf, samples, X_train_imp, y_train, X_test_imp, y_test)

LinearRegression trained on 2536 samples.
MSE_train: 13.9921
MSE_test: 18.1959
Training score:0.6785
Test score:0.6256


In [557]:
feature_importances = pd.DataFrame(clf.coef_,
                                   index = X_train.columns,
                                    columns=['coefficient']).sort_values('coefficient', ascending=False)

In [558]:
feature_importances

,coefficient
host_total_listings_count,1.846644e+12
review_scores_value,2.229004e+00
review_scores_cleanliness,1.825653e+00
review_scores_accuracy,1.127441e+00
review_scores_communication,7.326355e-01
review_scores_checkin,6.766968e-01
zipcode_98116,6.570053e-01
zipcode_98106,6.510086e-01
zipcode_98126,6.337585e-01
zipcode_98199,5.808411e-01


### Business Question 2: When and where are the most popular times of the year for rentals of Seattle homes?

For this question we are mainly looking at the calendar dataset. As seen during the Data Exploration stage, the dataset contains the listings, dates, availability and the price. 

To answer the question on the most popular times of the year for rentals, we will need to look at the dates and the availability of all listings.

But first, we need to clean the datasets of missing values.

In [562]:
#check missing values
calendar.isnull().sum()

listing_id         0
date               0
available          0
price         459028
dtype: int64

Good news is that the columns of our interest do not contain any missing values. We can drop the pricing data from this dataset as it is not of our interest.

In [566]:
calendar_clean=calendar.drop(columns='price')

In [567]:
#view sample dataset
calendar_clean.head(5)

,listing_id,date,available
0,241032,2016-01-04,t
1,241032,2016-01-05,t
2,241032,2016-01-06,f
3,241032,2016-01-07,f
4,241032,2016-01-08,f


Now the calendar dataset contains the listings with the dates and their availability, which is sufficient for us to determine when the most popular times of the year are.

Meanwhile, to get location data, we have to merge the calendar dataset with the listings dataset. In particular, we need to merge it with the location columns which include 'street', 'neighbourhood','neighbourhood_cleansed','neighbourhood_group_cleansed','city', 'state', 'zipcode','market','smart_location','country_code','country','latitude','longitude'. 

In [ ]:
#merge calendar dataset with listings dataset

305